## Setup

In [1]:
import pandas as pd
import numpy as np
import config

In [2]:
df = pd.read_csv(config.TRAIN_DIR + "taxonomy.csv", index_col = 0)
df.head()

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Abiotrophia_defectiva,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_fermentans,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_intestini,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00957,0.006907,0.0,0.0,0.0,0.0,0.009285
s__Actinomyces_graevenitzii,0.003491,0.000110,0.000004,0.000435,0.001297,0.000168,0.005996,0.000021,0.000039,0.000004,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Actinomyces_odontolyticus,0.000377,0.000005,0.000000,0.000014,0.000028,0.000107,0.000080,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000027


In [3]:
species = df.copy()
species[species < 0.00001] = 0
species.head()

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Abiotrophia_defectiva,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_fermentans,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_intestini,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00957,0.006907,0.0,0.0,0.0,0.0,0.009285
s__Actinomyces_graevenitzii,0.003491,0.00011,0.0,0.000435,0.001297,0.000168,0.005996,0.000021,0.000039,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Actinomyces_odontolyticus,0.000377,0.00000,0.0,0.000014,0.000028,0.000107,0.000080,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000027


In [4]:
with open(config.OUTPUT_DIR + "MH_species.txt") as f:
    MH_species = f.read().splitlines()
with open(config.OUTPUT_DIR + "MN_species.txt") as f:
    MN_species = f.read().splitlines()
MH_species

['s__Alistipes_senegalensis',
 's__Bacteroidales_bacterium_ph8',
 's__Bifidobacterium_adolescentis',
 's__Bifidobacterium_angulatum',
 's__Bifidobacterium_catenulatum',
 's__Lachnospiraceae_bacterium_8_1_57FAA',
 's__Sutterella_wadsworthensis']

In [5]:
MH_species_metagenome = species[species.index.isin(MH_species)]
MN_species_metagenome = species[species.index.isin(MN_species)]
display(MH_species_metagenome)
display(MN_species_metagenome)

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Alistipes_senegalensis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.001058,0.000000,0.000000,0.000712,0.000000,0.000000,0.000000,0.000000
s__Bacteroidales_bacterium_ph8,0.000000,0.000103,0.000212,0.000000,0.000763,0.000000,0.00000,0.000000,0.000000,0.000050,...,0.000000,0.000000,0.009711,0.004391,0.005546,0.060104,0.000000,0.007427,0.000000,0.002521
s__Bifidobacterium_adolescentis,0.022290,0.227322,0.090593,0.091049,0.070562,0.020331,0.17816,0.011900,0.022083,0.136260,...,0.001126,0.001084,0.016068,0.004492,0.013137,0.000000,0.013176,0.010072,0.035911,0.045419
s__Bifidobacterium_angulatum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Bifidobacterium_catenulatum,0.004416,0.000228,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000490,0.000000,...,0.000394,0.000489,0.000000,0.000640,0.002786,0.000000,0.000245,0.000000,0.001325,0.009873
s__Lachnospiraceae_bacterium_8_1_57FAA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.001437,0.000000,0.000000,...,0.001264,0.000895,0.000000,0.000371,0.000345,0.001208,0.000000,0.000000,0.000000,0.001000
s__Sutterella_wadsworthensis,0.006551,0.000183,0.001943,0.007306,0.000000,0.000000,0.00000,0.002993,0.004440,0.000773,...,0.000000,0.000000,0.000000,0.005554,0.016498,0.007777,0.005217,0.000000,0.001308,0.001799


,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Anaerotruncus_colihominis,0.000000,0.000000,0.000000,0.000000,0.000932,0.000000,0.000000,0.000000,0.000125,0.0,...,0.000031,0.000000,0.000016,0.000373,0.000233,0.000290,0.000000,0.000000,0.000173,0.000463
s__Atopobium_parvulum,0.000000,0.000000,0.000000,0.000075,0.000000,0.000000,0.000458,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Bifidobacterium_dentium,0.000016,0.000000,0.000000,0.000000,0.000000,0.000044,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000601,0.000000
s__Blautia_producta,0.000000,0.000000,0.000000,0.000000,0.000249,0.000000,0.000000,0.000000,0.000350,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__candidate_division_TM7_single_cell_isolate_TM7c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Clostridiales_bacterium_1_7_47FAA,0.000000,0.000000,0.000000,0.000000,0.000089,0.000194,0.000000,0.000229,0.000045,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Clostridium_asparagiforme,0.000000,0.000000,0.000000,0.000000,0.000509,0.000000,0.000000,0.001037,0.000083,0.0,...,0.000000,0.000000,0.000000,0.000080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Clostridium_bolteae,0.000000,0.000331,0.000651,0.000000,0.000887,0.000625,0.000000,0.004419,0.002272,0.0,...,0.000045,0.000032,0.000000,0.000353,0.000019,0.000000,0.000000,0.000000,0.000000,0.000334
s__Clostridium_citroniae,0.000000,0.000000,0.000104,0.000000,0.000000,0.000255,0.000000,0.000162,0.000056,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000045,0.000000,0.000000,0.000000


In [6]:
def shannon(mat):
    shannoned = np.log(mat) * mat * -1
    shannoned = shannoned.fillna(0)
    sums = shannoned.sum(axis=0)
    return pd.DataFrame(sums).T

In [7]:
MH_shannon = shannon(MH_species_metagenome)
MN_shannon = shannon(MN_species_metagenome)
MN_shannon

<ipython-input-6-674b95fac211>:2: RuntimeWarning: divide by zero encountered in log
  shannoned = np.log(mat) * mat * -1


,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
0,0.325973,0.034442,0.097553,0.086491,0.305192,0.125423,0.350531,0.247446,0.188338,0.0,...,0.016895,0.013565,0.095817,0.046305,0.031242,0.002852,0.019083,0.005824,0.051923,0.077803


In [8]:
def richness(df):
    frame = pd.DataFrame((df > 0).sum()).T
    frame.index = ['Richness']
    return frame

In [9]:
R_MH = richness(MH_species_metagenome)
R_MN = richness(MN_species_metagenome)
R_MH

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Richness,3,4,3,2,2,1,1,3,3,3,...,3,3,3,5,5,4,3,2,3,5


In [10]:
# From paper
MH_prime = 7
MN_prime = 31

In [13]:
psi_MH = pd.DataFrame((R_MH / MH_prime).values * MH_shannon.values, 
                      
                      columns=MH_shannon.columns, index=["psi_MH"])
psi_MN = pd.DataFrame((R_MN / MN_prime).values * MN_shannon.values, 
                      columns=MN_shannon.columns, index=["psi_MN"])
psi_MH

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
psi_MH,0.060715,0.194964,0.099201,0.072606,0.055016,0.011314,0.043906,0.034085,0.047993,0.118983,...,0.008215,0.007461,0.050841,0.060427,0.123268,0.125737,0.037072,0.023636,0.058685,0.1567


In [14]:
# these psi's don't use shannon term
psi_MH2 = pd.DataFrame((R_MH / MH_prime).values,
                      columns=R_MH.columns, index=["psi_MH"])
psi_MN2 = pd.DataFrame((R_MN / MN_prime).values,
                      columns=R_MN.columns, index=["psi_MN"])
psi_MH2

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
psi_MH,0.428571,0.571429,0.428571,0.285714,0.285714,0.142857,0.142857,0.428571,0.428571,0.428571,...,0.428571,0.428571,0.428571,0.714286,0.714286,0.571429,0.428571,0.285714,0.428571,0.714286


In [15]:
# using the psi's without shannon term
GMHI = pd.DataFrame(np.log10((psi_MH2+0.00001).values / (psi_MN2+0.00001).values), 
                    columns=psi_MH.columns, index=["GMHI"]).T
GMHI

,GMHI
SAMD00036192,-0.080733
SAMD00036193,0.294074
SAMD00036194,-0.080733
SAMD00036197,-0.198829
SAMD00036204,-0.283148
...,...
SAMN07534852,0.947234
SAMN07534870,0.220288
SAMN07534977,0.947174
SAMN07534978,0.081990


In [16]:
(GMHI > 0).sum()

GMHI    2614
dtype: int64

In [18]:
isHealthy = pd.read_csv(config.TRAIN_DIR + "isHealthy.csv", index_col=0)
isHealthy

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
isHealthy,True,True,False,True,True,True,False,True,True,False,...,False,False,True,True,False,True,True,True,True,True


In [19]:
def balanced_accuracy(GMHI, y):
    full = pd.concat([GMHI, y], axis=1)
    healthies = full[full['isHealthy']].iloc[:, [0]]
    predictedHealthy = int((healthies > 0).sum())
    unhealthies = full[~full['isHealthy']].iloc[:, [0]]
    predictedNot = int((unhealthies < 0).sum())
    P_H = predictedHealthy / len(healthies)
    P_N = predictedNot / len(unhealthies)
    print(P_H, P_N)
    chi = 0.5 * (P_H + P_N)
    return chi
balanced_accuracy(GMHI, isHealthy.T)

0.7621396054628224 0.6417299824663939


0.7019347939646081

## Balanced Accuracy of 0.697 hurray